# Integrate Scanpy into Spatiology Workflow

In [ ]:
import os
import importlib
import logging
import pandas as pd
import scanpy as sc
import cv2
import numpy as np
from typing import Tuple, List, Dict, Optional, Union
import inspect
import sys
import SpatioloJI as SJ

## Run basic workflow of Scanpy
    - Set up directory paths for original data/saved data
    - Scanpy preprocessing and clustering
      - Normalization
      - Dimensionality reduction
      - Clustering
    - Visualization for leiden clusters
    - Scanpy annotation
      - Marker genes
      - Differentially expressed genes
    - Visualization for annotated leiden clusters
    - Save annotated and processed adata for original matrix

In [ ]:
# load original filtered data
data_read = './test_data/' # for raw data 
data_save = './test_data/' # for processed data or intermediate result
analysis_save = './test_analysis/' # for plots

counts = pd.read_csv(data_save+'filtered_exprMat_file.csv', index_col=0)
polygon_file = pd.read_csv(data_save+'filtered_polygons.csv', index_col=0)
coordinates = pd.read_csv(data_save+'filtered_coordinates.csv', index_col=0) # optional if you want to store coordinates

# scanpy
adata = AnnData(counts, obsm={'spatial':coordinates}, obs=cell_meta)
adata.layers["counts"] = adata.X.copy()
sc.pp.normalize_total(adata, inplace=True)
sc.pp.log1p(adata)
sc.pp.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=2)

# Visualization
## specify saving folder for scanpy
sc.settings.figdir = analysis_save

## color preparation for categorical data (check whether they are strings/numeric ) and visualization of UMAP
adata.obs['leiden'] = adata.obs['leiden'].astype('category')
adata.obs['leiden'] = adata.obs['leiden'].cat.reorder_categories([str(i) for i in range(len(adata.obs['leiden'].unique()))], ordered=True)
color_palette = plt.cm.tab20.colors  # choose a different palette
color_palette = list(color_palette)
color_palette = color_palette[:len(adata.obs.leiden.unique())]
sc.pl.umap(adata, color='leiden', save='_leiden_raw.png', palette=color_palette)
df_leiden_color = {i:[to_hex(j)] for i,j in zip(adata.obs['leiden'].cat.categories, color_palette)}
pd.DataFrame(df_leiden_color, index=['color_hex_code']).T.to_csv(data_save+'leiden_color_hex_code.csv')

adata.obs['fov'] = adata.obs['fov'].astype('category')
adata.obs['fov'] = adata.obs['fov'].cat.reorder_categories([i for i in fov_id], ordered=True)
color_palette = sns.color_palette("hls", len(adata.obs['fov'].unique()))  # choose a different palette
color_palette = list(color_palette)
sc.pl.umap(adata, color='fov', save='_fov_raw.png', palette=color_palette)
df_fov_color = {i:[to_hex(j)] for i,j in zip(adata.obs['fov'].cat.categories, color_palette) }
pd.DataFrame(df_fov_color, index=['color_hex_code']).T.to_csv(data_save+'fov_color_hex_code.csv')



## General Markers 
marker_genes = {
    "Endo":["PECAM2"],
    "Meso": ["CALB2"],
    "FB": ["PDPN", "ACTA2", "DCN"],
    "EOC": ["EPCAM"],
    "CD14+ Mono": ["FCN1", "CD14"],
    "CD16+ Mono": ["TCF7L2", "FCGR3A", "LYN"],
    # Note: DMXL2 should be negative
    "cDC2": ["CST3", "COTL1", "LYZ", "DMXL2", "CLEC10A", "FCER1A"],
    "NK": ["GNLY", "NKG7", "CD247", "FCER1G", "TYROBP", "KLRG1", "FCGR3A"],
    "ILC": ["ID2", "PLCG2", "GNLY", "SYNE1"],
    "Naive CD20+ B": ["MS4A1", "IL4R", "IGHD", "FCRL1", "IGHM"],
    # Note IGHD and IGHM are negative markers
    "B cells": [
        "MS4A1",
        "ITGB1",
        "COL4A4",
        "PRDM1",
        "IRF4",
        "PAX5",
        "BCL11A",
        "BLK",
        "IGHD",
        "IGHM",
    ],
    "Plasma cells": ["MZB1", "FNDC3B", "PRDM1", "IGKC", "JCHAIN"],
    "CD4+ T": ["CD4", "IL7R", "TRBC2"],
    "CD8+ T": ["CD8A", "CD8B", "GZMK", "GZMA", "CCL5", "GZMB", "GZMH", "GZMA"],
    "T naive": ["LEF1", "CCR7", "TCF7"],
    "pDC": ["GZMB", "IL3RA", "COBLL1", "TCF4"],
}
marker_genes_filtered = {
    cell_type: [gene for gene in genes if gene in adata.var_names]
    for cell_type, genes in marker_genes.items()
}
marker_genes_filtered = {cell_type: genes for cell_type, genes in marker_genes_filtered.items() if genes}

sc.pl.dotplot(adata, marker_genes_filtered, groupby="leiden", standard_scale="var", save='_general_markers.png')
sc.pl.dotplot(adata, ['Mean.PanCK','Max.PanCK', 'Mean.CD45', 'Max.CD45', 'Mean.CD3', 'Max.CD3'], groupby="leiden", standard_scale="var", save='_morphological_markers.png')

## Cell Markers
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon', n_genes=adata.shape[1])
sc.tl.dendrogram(adata,groupby='leiden')

sc.pl.rank_genes_groups_dotplot(adata, 
        n_genes = 5,
        values_to_plot="logfoldchanges",
        cmap='bwr',
        min_logfoldchange=0.5,
        colorbar_title='log fold change',title='leiden DE genes', save='_top5_marker.png')

adata.obs['big_cell_type'] = adata.obs['leiden'].map({
    '0':'IM',
    '1':'EOC',
    '2':'IM',
    '3':'FB',
    '4':'IM',
    '5':'IM',
    '6':'FB',
    '7':'IM',
    '8':'IM',
    '9':'IM',
    '10':'IM',
    '11':'IM',
    '12':'IM',
    '13':'IM',
    '14':'EOC',
    '15':'IM',
    '16':'IM',
    '17':'IM',
    '18':'IM',
    '19':"EOC"
})

adata.obs['small_cell_type'] = [i+'_'+j for i,j in zip(adata.obs['big_cell_type'].tolist(), adata.obs['leiden'].tolist())]
adata.obs[['big_cell_type','small_cell_type']]


## color preparation
adata.obs['big_cell_type'] = adata.obs['big_cell_type'].astype('category')
adata.obs['big_cell_type'] = adata.obs['big_cell_type'].cat.reorder_categories(['EOC', 'FB', 'IM'],ordered=True)
color_palette = plt.cm.Set1.colors  # choose a different palette
color_palette = list(color_palette)
color_palette = color_palette[:len(adata.obs['big_cell_type'].unique())]
df_big_cell_type_color = {i:[to_hex(j)] for i,j in zip(adata.obs['big_cell_type'].cat.categories, color_palette) }
pd.DataFrame(df_big_cell_type_color, index=['color_hex_code']).T.to_csv(data_save+'big_cell_type_color_hex_code.csv')

## barplot
count_tab = pd.crosstab(adata.obs['big_cell_type'], adata.obs['fov'])
proportion_df = count_tab.div(count_tab.sum(axis=0), axis=1)
plt.figure(figsize=(6,5))
proportion_df.T.plot(kind='bar', stacked=True, color=[to_hex(i) for i in color_palette])
plt.ylabel("Cell Count Proportion")
plt.title("Stacked Barplot of Cell Types per fov")
plt.legend(title="Cell Type", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig(analysis_save+'stacked_barplot_big_CT_fov.png')
plt.show()


count_tab = pd.crosstab(adata.obs['big_cell_type'], adata.obs['fov'])
proportion_df = count_tab.div(count_tab.sum(axis=1), axis=0)
plt.figure(figsize=(6,5))
proportion_df.plot(kind='bar', stacked=True, color=np.array([i for i in df_fov_color.values()]).flatten().tolist())
plt.ylabel("fov Proportion")
plt.title("Stacked Barplot of fov per Cell Types")
plt.legend(title="fov", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig(analysis_save+'stacked_barplot_fov_big_CT.png')
plt.show()

# save adata for original data
adata.write(data_save+'filtered_adata.h5ad')




## Imputation
    - Impute normalized expression matrix utilizing MAGIC (https://github.com/KrishnaswamyLab/MAGIC)
    - Scanpy preprocessing and clustering
      - Normalization
      - Dimensionality reduction
      - Clustering
    - Save processed data for inferred matrix

In [ ]:
# imputation 
count = adata.X.copy()
count = pd.DataFrame(data=count, index=adata.obs.index.tolist(), columns = adata.var.index.tolist())
cell_meta = adata.obs
coordinates = adata.obsm['spatial']
magic_operator = magic.MAGIC()
count = magic_operator.fit_transform(count, genes='all_genes')
# scanpy workflow
adata = AnnData(count, obsm={'spatial':coordinates}, obs=cell_meta)
sc.pp.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=2)

adata.write(data_save+'filtered_magic_adata.h5ad')

## Instantiate SpatioloJI object
    - Load necessary data
      - Filtered polygon file
      - Filtered cell meta file
      - Filtered and processed adata
      - Filtered fov position file
      - Filtered images
    - Get&save SP_JI object

In [2]:
# load data
data_read = './test_data/' # for raw data 
data_save = './test_data/' # for processed data or intermediate result
analysis_save = './test_analysis/' # for plots
image_dir = './test_data/image/'

adata = sc.read(data_save+'filtered_adata.h5ad')
cell_meta = pd.read_csv(data_save+'filtered_metadata_file.csv', index_col=0)
polygon_file = pd.read_csv(data_save+'filtered_polygons.csv', index_col=0)
fov_id = [str(i) for i in range(1,23) if i not in [3, 20]]

image = cv2.imread(image_dir+"CellComposite_F001.jpg", cv2.IMREAD_COLOR)
xlim_image = image.shape[1]
ylim_image = image.shape[0]


fov_positions = pd.read_csv(data_save+"fov_positions.csv")
# fov_positions.set_index("fov", inplace=True)
min_x = fov_positions["x_global_px"].min()
min_y = fov_positions["y_global_px"].min()

cell_meta['fov'] = cell_meta['fov'].astype(str).tolist()
polygon_file['fov'] = polygon_file['fov'].astype(str).tolist()
adata.obs['fov'] = adata.obs['fov'].astype(str).tolist()

# get&save sp_ji
sp_ji = SJ.data.Spatioloji(
    polygons=polygon_file,
    cell_meta=cell_meta,
    adata=adata,
    fov_positions=fov_positions,
    images_folder='./test_data/image',
    prefix_img='CellComposite_F',  # Default, can be changed
    img_format='jpg'               # Default, can be changed
)

sp_ji.to_pickle(data_save+'sp_ji.pkl')